In [1]:
import mltable
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("automobile_prices_99", version="1")

tbl = mltable.load(f'azureml:/{data_asset.id}')

df = tbl.to_pandas_dataframe()
df

Found the config file in: /config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Timeout was exceeded in force_flush().
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is 

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NA,ferrari,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,113495
1,3,NA,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
2,3,NA,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
3,1,NA,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
4,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845
202,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045
203,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485
204,-1,95,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.4,23.0,106,4800,26,27,22470


In [2]:
# Save modified dataframe to a local folder as a CSV file
from azure.ai.ml.entities import Data


# duplicate and modify the dataframe
df_modified = df.copy()[["make","body-style","wheel-base","engine-size", "horsepower", "peak-rpm", "highway-mpg", "price"]]
df_modified

modified_filename = "automobile_prices_modified.csv"
modified_table_asset = "automobile_prices_modified_mltable5"
output_dir = "modified_data"
os.makedirs(output_dir, exist_ok=True)

# Step 1: Save the DataFrame to a folder
csv_path = os.path.join(output_dir, modified_filename)
df_modified.to_csv(csv_path, index=False)

# Step 2: Create MLTable YAML file in the same folder
with open(os.path.join(output_dir, "MLTable"), "w") as f:
    f.write(f"""paths:
  - file: ./{modified_filename}
transformations:
  - read_delimited:
      delimiter: ","
      encoding: "utf8"
      header: all_files_same_headers
""")
table_data = Data(
    path=output_dir,
    type="mltable",  # <-- Important: this sets it as table-type
    name=modified_table_asset,
    description=modified_table_asset,
)

# Register the new dataset
ml_client.data.create_or_update(table_data)


Uploading modified_data (0.02 MBs): 100%|██████████| 17391/17391 [00:00<00:00, 290077.35it/s]




Data({'path': 'azureml://subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourcegroups/mmdatascience04grp/workspaces/mmamlwksp04/datastores/workspaceblobstore/paths/LocalUpload/f90f966976062b025d34e87124a20694/modified_data/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': ['./automobile_prices_modified.csv'], 'type': 'mltable', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'automobile_prices_modified_mltable5', 'description': 'automobile_prices_modified_mltable5', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmdatascience04grp/providers/Microsoft.MachineLearningServices/workspaces/mmamlwksp04/data/automobile_prices_modified_mltable5/versions/4', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/dsvm-mem-optimized04/code/Users/mauro.minella/git_repos/ml_tests', 'creation_context': <azure.